In [1]:
import os
import csv
import shutil
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

C:\Users\Amith\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## creating metadata for male


# Path to the folder containing text files
text_folder_path = 'D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/txt/'

# Path to save the metadata.csv file
metadata_csv_path = 'metadata-male_40hrs.csv'

# Open metadata.csv file for writing
with open(metadata_csv_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter='|')
    
    # Write the header row
    csv_writer.writerow(['id', 'text', 'speaker'])
    
    # Iterate through each text file in the folder
    for filename in os.listdir(text_folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(text_folder_path, filename)
            
            # Read the text content from the file
            with open(file_path, 'r', encoding='utf-8') as text_file:
                text_content = text_file.read().strip()
            
            # Get the filename without extension
            filename_without_extension = os.path.splitext(filename)[0]
            
            # Assume male for all entries (you may modify this based on your actual data)
            speaker = 'male'
            
            # Write a row to the metadata.csv file
            csv_writer.writerow([filename_without_extension, text_content, speaker])

print(f"Metadata.csv file has been created at: {metadata_csv_path}")


Metadata.csv file has been created at: metadata-male_40hrs.csv


In [4]:
import pandas as pd
metadata = pd.read_csv('D:/THESIS/TTS_40HRS/Telugu_M/metadata-male_40hrs.csv',sep='|')
#metadata_female = pd.read_csv('metadata-female.csv',sep='|')
metadata.head()

,id,text,speaker
0,te_m_agri_00001,"జంతువుల వ్యవసాయం అంతర్గతంగా అసమర్థంగా ఉంటుంది,...",male
1,te_m_agri_00002,ఏడు ప్రధాన పెలికాన్ జాతులు ఉన్నాయి గోధుమ పెలిక...,male
2,te_m_agri_00003,"జిల్లాలోని జన్నారం, దండేపల్లి, చె న్నూర్, వేమన...",male
3,te_m_agri_00004,"ఆవిరి పీల్చడం మూలికా డికాక్షన్స్ సాజ్, యూకలిప్...",male
4,te_m_agri_00005,వీటిన్నింటినీ దృష్టిలో పెట్టుకుని ఇన్పుట్ సబ్స...,male


In [5]:
metadata.to_csv('D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/metadata.csv', sep='|', index=False, header=False)

In [6]:
df = pd.read_csv('D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/metadata.csv', sep='|', names=['id', 'text', 'speaker'])
print(df.shape)
df.head()

(16835, 3)


,id,text,speaker
0,te_m_agri_00001,"జంతువుల వ్యవసాయం అంతర్గతంగా అసమర్థంగా ఉంటుంది,...",male
1,te_m_agri_00002,ఏడు ప్రధాన పెలికాన్ జాతులు ఉన్నాయి గోధుమ పెలిక...,male
2,te_m_agri_00003,"జిల్లాలోని జన్నారం, దండేపల్లి, చె న్నూర్, వేమన...",male
3,te_m_agri_00004,"ఆవిరి పీల్చడం మూలికా డికాక్షన్స్ సాజ్, యూకలిప్...",male
4,te_m_agri_00005,వీటిన్నింటినీ దృష్టిలో పెట్టుకుని ఇన్పుట్ సబ్స...,male


In [7]:
df_train, df_test = train_test_split(df, test_size=0.01,random_state=0)
print(len(df_train), len(df_test))

16666 169


In [8]:
df_train.to_csv('D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/metadata_train.csv', sep='|', index=False, header=False)
df_test.to_csv('D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/metadata_test.csv', sep='|', index=False, header=False)

In [10]:
import os
import librosa

def process_audio_directory(directory_path, n_mfcc=20, n_fft=1024, hop_length=512):
  """
  Processes audio files in a directory, extracting MFCC features and identifying files with zero signal length.

  Args:
      directory_path: Path to the directory containing audio files.
      n_mfcc: Number of Mel-frequency cepstral coefficients to extract (default: 20).
      n_fft: Window size for FFT (default: 2048).
      hop_length: Hop length for MFCC computation (default: 512).

  Returns:
      A tuple containing:
          - features: A list of MFCC features for valid audio files (empty if no valid files).
          - empty_signal_files: A list of paths to files with zero signal length.
  """

  features = []
  empty_signal_files = []

  for filename in os.listdir(directory_path):
    if filename.endswith(".wav"):  # Check for WAV files only
      filepath = os.path.join(directory_path, filename)

      try:
        signal, sr = librosa.load(filepath, sr=None)
        if signal is None or len(signal) == 0:
          empty_signal_files.append(filepath)
          continue
        else:
          feature = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
          features.append(feature)
      except (librosa.util.exceptions.ParameterError, OSError) as e:
        print(f"Error processing file '{filepath}': {e}")
        continue

  return features, empty_signal_files

# Example usage
directory_path = "D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/resample_wav/"
features, empty_signal_files = process_audio_directory(directory_path)

if features:
  print("Extracted MFCC features for valid audio files.")
else:
  print("No valid audio files found in the directory.")

if empty_signal_files:
  print("Files with zero signal length:")
  for file in empty_signal_files:
    print(file)
else:
  print("No files with zero signal length found.")


C:\Users\Amith\AppData\Local\Temp\ipykernel_24084\3226818909.py:28: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, sr = librosa.load(filepath, sr=None)
C:\Users\Amith\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error processing file 'D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/resample_wav/te_m_education_01162.wav': [Errno 2] No such file or directory: 'D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/resample_wav/te_m_education_01162.wav'
Error processing file 'D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/resample_wav/te_m_education_01163.wav': [Errno 2] No such file or directory: 'D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/resample_wav/te_m_education_01163.wav'
Error processing file 'D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/resample_wav/te_m_education_01164.wav': [Errno 2] No such file or directory: 'D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/resample_wav/te_m_education_01164.wav'
Error processing file 'D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/resample_wav/te_m_education_01165.wav': [Errno 2] No such file or directory: 'D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/resample_wav/te_m_education_01165.wav'
Error processing file 'D:/THESIS/TTS_40HRS/Telugu_M/Telugu_M/resample_wav/te_m_education_01166.wav': [Errno 2] No such file or directory